# Strands Agents クイックスタート

Strands Agents は、AWS サービスと連携して複雑なタスクを実行できる AI エージェントを構築するための強力なフレームワークです。このクイックスタートガイドは、初めての Strands エージェントの作成に役立ちます。

## 前提条件

- Python 3.10 以降
- 適切な権限が設定された AWS アカウント
- Python プログラミングの基礎知識

さあ、始めましょう！

In [ ]:
# pipを使ってStrandsをインストールする

!pip install strands-agents strands-agents-tools


## 最初のエージェントの作成

必要なエージェントコンポーネントの概要を確認しましょう。

### 1. シンプルなエージェントの作成:

これにより、デフォルトのモデルプロバイダー [Amazon Bedrock](https://aws.amazon.com/bedrock/) とデフォルトのモデル Claude 3.7 Sonnet を使用して、AWS セットアップのリージョンにエージェントが作成されます。エージェントは呼び出されるのと同じローカル環境で実行されますが、Amazon Bedrock モデルは AWS アカウントで実行され、エージェントはクラウドアカウント内のモデルを呼び出します。アーキテクチャは次のようになります。

<div style="text-align:center">
    <img src="images/simple_agent.png" width="75%" />
</div>

In [ ]:
from strands import Agent

# エージェントを初期化
agent = Agent(
    model = "anthropic.claude-3-sonnet-20240229-v1:0",    # model パラメータを省略すると Claude 3.7 Sonnet が使用される
    system_prompt="あなたは簡潔な回答を提供する親切なアシスタントです。"
)

# エージェントにメッセージを送信
response = agent("こんにちは！何ができますか？")
print(response)

### 2. エージェントにツールを追加する:

[strands-agents-tools](https://github.com/strands-agents/tools) リポジトリには、インポート可能な組み込みツールがいくつか用意されています。また、`@tool` デコレータを使用してカスタムツールを作成することもできます。組み込みツールとカスタムツールの両方を備えたエージェントを作成できます。例えば、電卓の組み込みツールと天気予報を取得するカスタムツールを追加すると、次のアーキテクチャになります。
<div style="text-align:center">
<img src="images/agent_with_tools.png" width="75%" />
</div>

このアーキテクチャを実装すると、次のようになります。

In [ ]:
from strands import Agent, tool
from strands_tools import calculator # 計算ツールをインポートする

# カスタムツールを作成する
@tool
def weather():
    """ 天気を取得 """ # ダミー実装
    return "晴れ"

agent = Agent(
    model = "anthropic.claude-3-sonnet-20240229-v1:0", 
    tools=[calculator, weather],    # ツールを指定
    system_prompt="あなたは親切なアシスタントです。簡単な数学計算ができ、天気を教えることができます。"
)

response = agent("今日の天気はどうですか？")

print(response)

### ツールの直接呼び出し

アプリケーションによっては、ツールを直接呼び出すことが重要です。例えば、ツールをデバッグしたり、エージェントのナレッジに顧客情報を事前に入力したり、別のツール内でツールを使用したりする場合などです。Strandsでは、エージェントの「tool」メソッドにツール名を続けて使用することで、これを実行できます。

In [ ]:
# 微分を計算
agent.tool.calculator(expression="sin(x)", mode="derive", wrt="x", order=2)


### 3. ログレベルと形式の変更:

Strands SDK は、Python の標準 `logging` モジュールを使用して、その操作を可視化します。

Strands Agents SDK は、シンプルなログ記録アプローチを実装しています。

1. **モジュールレベルのロガー**: SDK 内の各モジュールは、階層的なログ記録に関する Python のベストプラクティスに従い、logging.getLogger(__name__) を使用して独自のロガーを作成します。
2. **ルートロガー**: SDK 内のすべてのロガーは「strands」ルートロガーの子であるため、SDK 全体のログ記録を簡単に設定できます。
3. **デフォルトの動作**: デフォルトでは、SDK はハンドラーやログレベルを設定しません。そのため、アプリケーションのログ記録設定と統合できます。

Strands Agents SDK のログ記録を有効にするには、「strands」** ロガーを設定します。たとえばデバッグ中にログ レベルを変更したり、ログ形式を変更したりする場合は、次のようにロガー構成を設定できます。

In [ ]:
import logging
from strands import Agent

# Strandsデバッグログレベルを有効化
logging.getLogger("strands").setLevel(logging.DEBUG) # またはlogging.ERROR

# ログ形式を設定し、ログをstderrにストリーム
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

agent = Agent(model = "anthropic.claude-3-sonnet-20240229-v1:0")
agent("こんにちは！")


### 4. モデルプロバイダー

デフォルトのモデルプロバイダーは [Amazon Bedrock](https://aws.amazon.com/bedrock/) で、現在の AWS 環境のリージョンにおけるデフォルトのモデルは Claude 3.7 Sonnet です。

Amazon Bedrock では、モデル ID 文字列を直接指定することで、別のモデルを指定することもできます。

In [ ]:
from strands import Agent

agent = Agent(model = "anthropic.claude-3-sonnet-20240229-v1:0")
print(agent.model.config)


モデル構成をさらに制御するには、`BedrockModel` プロバイダー インスタンスを作成します。

In [ ]:
import boto3
from strands import Agent
from strands.models import BedrockModel

# Create a BedrockModel
bedrock_model = BedrockModel(
    model = "anthropic.claude-3-sonnet-20240229-v1:0",
    region_name='us-east-1',
    temperature=0.3,
)

agent = Agent(model=bedrock_model)


利用可能なモデル プロバイダーの詳細については、[モデル プロバイダー クイックスタート ページ](https://strandsagents.com/0.1.x/user-guide/quickstart/#model-providers) を参照してください。


**おめでとうございます!!これで、Strands を使用して簡単なエージェントを構築する方法を学習しました!!**

## [オプション] タスク特化型エージェント - RecipeBot を作成しましょう 🍽️

タスク特化型エージェントの実用的な例を作成しましょう。レシピを推奨し、料理に関する質問に答える「RecipeBot」を作成します。さあ、始めましょう！

作成するものは以下のとおりです。

<div style="text-align:center">
    <img src="images/interactive_recipe_agent.png" width="75%" />
</div>

In [ ]:
# 必要なパッケージをインストールする
%pip install duckduckgo-search # まだインストールしていない場合は、strands-agents strands-agents-toolsもインストールしてください。

In [ ]:
from strands import Agent, tool
from duckduckgo_search import DDGS
from duckduckgo_search.exceptions import RatelimitException, DuckDuckGoSearchException
import logging

# ログを設定
logging.getLogger("strands").setLevel(logging.INFO)

# ウェブ検索ツールを定義
@tool
def websearch(keywords: str, region: str = "us-en", max_results: int | None = None) -> str:
    """最新情報を取得するためにウェブを検索します
    Args:
        keywords (str): 検索クエリのキーワード
        region (str): 検索地域: wt-wt, us-en, uk-en, ru-ru, など
        max_results (int | None): 返す結果の最大数
    Returns:
        検索結果の辞書のリスト
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "結果が見つかりませんでした。"
    except RatelimitException:
        return "RatelimitException: 短時間待ってから再試行してください。"
    except DuckDuckGoSearchException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"

In [ ]:
# レシピアシスタントエージェントを作成
recipe_agent = Agent(
    model = "anthropic.claude-3-sonnet-20240229-v1:0",
    system_prompt="""あなたはRecipeBot、親切な料理アシスタントです。
    材料に基づいてレシピを見つけ、料理の質問に答えるお手伝いをします。
    ユーザーが材料について言及したり、料理情報を調べたりする際には、websearchツールを使用してレシピを見つけてください。""",
    tools=[websearch], # 上で作成したwebsearchツールをインポート
)

In [ ]:
response = recipe_agent("鶏肉とブロッコリーを使ったレシピを提案してください。")

print(f"メトリクス : {response.metrics}") # オプションですが、推奨されます。

# その他の例:
# response = recipe_agent("キヌアはどのように調理すればいいですか？")
# response = recipe_agent("エビパスタで白ワインを代用するにはどうすればいいでしょうか？")
# response = recipe_agent("アスパラガスの健康効果は何ですか?")
print(response)

#### これで完了です！わずか数行のコードで、ツールを備えたユースケースエージェントが実行可能になりました🥳。

より詳細なクイックスタートガイドについては、[Strands ドキュメント](https://strandsagents.com/0.1.x/user-guide/quickstart/)をご覧ください。

### [オプション] CLI 経由で RecipeBot を実行する:
`02_simple_interactive_usecase/recipe_bot.py` で提供されている Python スクリプトを使用して、コマンドライン（SageMaker Studio のターミナルなど）からエージェントを対話モードで実行できます。これにより、CLI 経由でメッセージの送信やレスポンスの受信など、エージェントとより動的にやり取りできるようになります。
エージェントを対話モードで実行するには、ターミナルで以下のコマンドを実行します。

```cli
cd dev_gen_ai_app_on_aws/StrandsAgentExamples/02-simple-interactive-usecase/
pip install -r requirements.txt
python recipes_bot.py
```

これにより、コマンドラインインターフェース (CLI) 経由でボットと対話できるようになります。